# Quizz Video Generator

## Introduction

In [11]:
"""Experimental module for subtitles support."""

import re

import numpy as np

from moviepy.decorators import convert_path_to_string
from moviepy.tools import convert_to_seconds
from moviepy.video.VideoClip import TextClip, VideoClip


class SubtitlesClipMine(VideoClip):
    """A Clip that serves as "subtitle track" in videos.

    One particularity of this class is that the images of the
    subtitle texts are not generated beforehand, but only if
    needed.

    Parameters
    ----------

    subtitles
        Either the name of a file as a string or path-like object, or a list

    font
        Path to a font file to be used. Optional if make_textclip is provided.

    make_textclip
        A custom function to use for text clip generation. If None, a TextClip
        will be generated.

        The function must take a text as argument and return a VideoClip
        to be used as caption

    encoding
        Optional, specifies srt file encoding.
        Any standard Python encoding is allowed (listed at
        https://docs.python.org/3.8/library/codecs.html#standard-encodings)

    Examples
    --------

    .. code:: python

        from moviepy.video.tools.subtitles import SubtitlesClip
        from moviepy.video.io.VideoFileClip import VideoFileClip
        generator = lambda text: TextClip(text, font='./path/to/font.ttf',
                                        font_size=24, color='white')
        sub = SubtitlesClip("subtitles.srt", make_textclip=generator, encoding='utf-8')
        myvideo = VideoFileClip("myvideo.avi")
        final = CompositeVideoClip([clip, subtitles])
        final.write_videofile("final.mp4", fps=myvideo.fps)

    """

    def __init__(self, subtitles, font=None, make_textclip=None, encoding=None):
        VideoClip.__init__(self, has_constant_size=False)

        if not isinstance(subtitles, list):
            # `subtitles` is a string or path-like object
            subtitles = file_to_subtitles(subtitles, encoding=encoding)

        # subtitles = [(map(convert_to_seconds, times), text)
        #              for times, text in subtitles]
        self.subtitles = subtitles
        self.textclips = dict()

        self.font = font

        if make_textclip is None:
            if self.font is None:
                raise ValueError("Argument font is required if make_textclip is None.")
            
            def make_textclip(txt):
                txt_clip = TextClip(text=txt, font=self.font, font_size=60, size=(700, None), color='white',method='caption',text_align="center",stroke_color="black",stroke_width=10)  
                #txt_clip = txt_clip.with_position(("center", "bottom"))
                return txt_clip

        self.size = (1080,None)  # Modiff
        self.make_textclip = make_textclip
        self.start = 0
        self.duration = max([tb for ((ta, tb), txt) in self.subtitles])
        self.end = self.duration
        

        def add_textclip_if_none(t):
            """Will generate a textclip if it hasn't been generated asked
            to generate it yet. If there is no subtitle to show at t, return
            false.
            """
            sub = [
                ((text_start, text_end), text)
                for ((text_start, text_end), text) in self.textclips.keys()
                if (text_start <= t < text_end)
            ]
            if not sub:
                sub = [
                    ((text_start, text_end), text)
                    for ((text_start, text_end), text) in self.subtitles
                    if (text_start <= t < text_end)
                ]
                if not sub:
                    return False
            sub = sub[0]
            if sub not in self.textclips.keys():
                self.textclips[sub] = self.make_textclip(sub[1])

            return sub

        def frame_function(t):
            sub = add_textclip_if_none(t)
            return self.textclips[sub].get_frame(t) if sub else np.array([[[0, 0, 0]]])

        def make_mask_frame(t):
            sub = add_textclip_if_none(t)
            return self.textclips[sub].mask.get_frame(t) if sub else np.array([[0]])

        self.frame_function = frame_function
        hasmask = bool(self.make_textclip("T").mask)
        self.mask = VideoClip(make_mask_frame, is_mask=True) if hasmask else None



@convert_path_to_string("filename")
def file_to_subtitles(filename, encoding=None):
    """Converts a srt file into subtitles.

    The returned list is of the form ``[((start_time,end_time),'some text'),...]``
    and can be fed to SubtitlesClip.

    Only works for '.srt' format for the moment.
    """
    times_texts = []
    current_times = None
    current_text = ""
    with open(filename, "r", encoding=encoding) as file:
        for line in file:
            times = re.findall("([0-9]*:[0-9]*:[0-9]*,[0-9]*)", line)
            if times:
                current_times = [convert_to_seconds(t) for t in times]
            elif line.strip() == "":
                times_texts.append((current_times, current_text.strip("\n")))
                current_times, current_text = None, ""
            elif current_times:
                current_text += line
    return times_texts

### Imports

In [12]:
import os
import json
import glob
import random
from datetime import timedelta
from PIL import Image

from classes.APIs import *
from classes.utils import Speech

## Generate questions

In [13]:
#print(generate_text("kylian mbappé"))

# Create class Project

In [14]:
from moviepy import CompositeVideoClip,VideoFileClip,VideoClip,vfx # Vidéo
from moviepy.video.tools.subtitles import SubtitlesClip # Sous Titres
from moviepy import ImageClip,TextClip # Image
from moviepy import vfx,AudioFileClip,concatenate_audioclips,CompositeAudioClip # Audio
from moviepy.video.fx import MultiplySpeed
from moviepy.audio.fx import MultiplyVolume



class Project():
    def __init__(self, project_name,speech, subject = None, output_path=None):
        self.project_name = project_name
        self.subject = subject if subject else project_name
        self.output_path = output_path if output_path else os.path.join("outputs", project_name)
        self.paths = self.create_dirs()
        self.speech = speech
        self.create_subtitles()
        
    def create_dirs(self):
        
        if os.path.exists(self.output_path):
            return
        os.makedirs(self.output_path, exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "audios"), exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "final"), exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "background"), exist_ok=True)
        return {
            "default": self.output_path,
            "audios": os.path.join(self.output_path, "audios"),
            "final": os.path.join(self.output_path, "final"),
            "background": os.path.join(self.output_path, "background")
        }
    def create_subtitles(self):
        # Write the hashtags to a .txt file
        with open(os.path.join(self.paths["final"], "hashtags.txt"), "w") as file:
            file.write(self.speech.text["hashtags"])

        print("File 'hashtags.txt' has been created!")
    
    def choose_background(self):
        png_files = glob.glob(os.path.join(self.paths["background"], "*.png"))
        square_png_files = []
        smallest_diff = 0

        for file_path in png_files:
            with Image.open(file_path) as img:
                # Obtenir les dimensions de l'image
                width, height = img.size

                # Vérifier si l'image est carrée (largeur == hauteur)
                if width == height:
                    # Ajouter le chemin de l'image carrée à la liste
                    square_png_files.append(file_path)
                    return file_path
                
                else:
                    # Calculer la différence entre largeur et hauteur
                    diff = width/height
                    
                    # Si l'image est plus proche d'un carré, on la garde
                    if abs(diff - 1) < abs(smallest_diff - 1):  # Vérifie la proximité avec 1
                        smallest_diff = diff
                        closest_image = file_path

        return closest_image
    
    def random_music(self):
        # Répertoire contenant les fichiers audio .mp3
        audio_dir = "..\musique"  # Remplace par le chemin de ton répertoire

        # Liste tous les fichiers dans le répertoire audio et garde seulement ceux avec l'extension .mp3
        audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.mp3')]

        # Vérifie s'il y a des fichiers .mp3 dans le répertoire
        if audio_files:
            # Choisit un fichier audio au hasard
            random_audio_file = random.choice(audio_files)
            
            # Crée le chemin complet du fichier audio choisi
            audio_path = os.path.join(audio_dir, random_audio_file)
            
            # Charge l'audio dans MoviePy
            musique = AudioFileClip(audio_path)

            print(f"\nFichier audio choisi : {random_audio_file}\n")
            return musique
            
        else:
            print("Aucun fichier audio .mp3 trouvé dans le répertoire.")
            return None
        
    def random_video(self):
        # Répertoire contenant les fichiers vidéo .mp4
        video_dir = "..\video_satisfaisante"  # Remplace par le chemin de ton répertoire

        # Liste tous les fichiers dans le répertoire vidéo et garde seulement ceux avec l'extension .mp4
        video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]

        # Vérifie s'il y a des fichiers .mp4 dans le répertoire
        if video_files:
            # Choisit un fichier vidéo au hasard
            random_video_file = random.choice(video_files)
            
            # Crée le chemin complet du fichier vidéo choisi
            video_path = os.path.join(video_dir, random_video_file)
            
            # Charge la vidéo dans MoviePy
            video_clip = VideoFileClip(video_path)

            print(f"\nFichier vidéo choisi : {random_video_file}\n")
            return video_clip
        else:
            print("\nAucune vidéo MP4 trouvée dans le répertoire spécifié.\n")
            return None

    def create_srt(self, duration=5, min_duration=1, max_length=50):
        """
        Crée un fichier .srt à partir d'un texte, en divisant par les ponctuations ".", "!", "?" et
        en calculant la durée des timestamps en fonction du nombre de caractères par phrase.
        
        :param duration: Durée totale de la vidéo en secondes.
        :param min_duration: Durée minimale d'un sous-titre (en secondes).
        :param max_length: Longueur maximale d'une phrase avant de la couper (en caractères).
        """
        def split_long_line(line, max_length):
            """
            Découpe une phrase en plusieurs morceaux, chacun respectant la longueur maximale.
            La coupure se fait au niveau des espaces pour ne pas couper les mots.
            """
            if len(line) <= max_length:
                return [line]
            
            words = line.split()
            chunks, current_chunk = [], ""

            for word in words:
                # Si le mot ajouté dépasse la longueur max, stocker le chunk actuel et recommencer un nouveau
                if len(current_chunk) + len(word) + 1 > max_length:  # +1 pour l'espace
                    chunks.append(current_chunk.strip())
                    current_chunk = word  # Commencer une nouvelle ligne avec le mot actuel
                else:
                    current_chunk += (" " + word if current_chunk else word)
            
            # Ajouter le dernier chunk s'il reste du texte
            if current_chunk:
                chunks.append(current_chunk.strip())
            
            return chunks


        # Combiner le texte de l'introduction, du contenu principal et de la conclusion
        text = self.speech.text["intro"] + " " + self.speech.text["text"] + " " + self.speech.text["outro"]

        # Diviser le texte en phrases avec ".", "!" et "?"
        lines = re.split(r'(?<=[.!?])\s+', text)  # Garde le séparateur, puis divise sur l'espace suivant
        lines = [line.strip() for line in lines if line.strip()]  # Nettoyer et enlever les phrases vides

        # Découper les phrases trop longues en plusieurs morceaux
        split_lines = []
        for line in lines:
            split_lines.extend(split_long_line(line, max_length))

        # Calculer le nombre total de caractères dans toutes les phrases
        total_characters = sum(len(line) for line in split_lines)

        # Ouvrir le fichier de sortie en mode écriture
        with open(os.path.join(self.paths["audios"], "subtitles.srt"), "w", encoding="utf-8") as f:
            current_time = 0  # Timestamp de début pour la première phrase

            for i, line in enumerate(split_lines):
                # Calculer la durée de chaque segment en fonction du nombre de caractères
                segment_duration = max((len(line) / total_characters) * duration, min_duration)

                # Calculer le timestamp de début et de fin pour chaque ligne
                start_time = str(timedelta(seconds=current_time))
                end_time = str(timedelta(seconds=current_time + segment_duration))

                # Formatter les timestamps au format SRT (hh:mm:ss,ms)
                start_time = start_time.split(".")[0] + ",000"
                end_time = end_time.split(".")[0] + ",000"

                # Écrire dans le fichier au format SRT
                f.write(f"{i + 1}\n")  # Numéro du sous-titre
                f.write(f"{start_time} --> {end_time}\n")  # Timestamp
                f.write(f"{line.strip()}\n\n")  # Contenu du sous-titre

                # Mettre à jour le temps courant
                current_time += segment_duration    
                
    def combined_audio(self):
        
        audio1 = AudioFileClip(os.path.join(self.paths["audios"], "intro.mp3"))
        audio2 = AudioFileClip(os.path.join(self.paths["audios"], "history.mp3"))
        audio3 = AudioFileClip(os.path.join(self.paths["audios"], "outro.mp3"))
        
        # Combiner les fichiers audio
        concatenate_audio = concatenate_audioclips([audio1, audio2, audio3])

        # Augmenter la vitesse de l'audio final (1.2x)
        #concatenate_audio = MultiplySpeed(factor=1.2).apply(concatenate_audio)

        musique = self.random_music()
        if musique is None:
            return None
        musique = musique.subclipped(0,concatenate_audio.duration)

        # gérer le son
        musique_faible = musique.with_effects([MultiplyVolume(0.025)]) # réduire le volume de la musique à 2,5 %

        # Superposer les deux audios
        combined_audio = CompositeAudioClip([concatenate_audio, musique_faible])      
        
        # Exporter l'audio combiné
        combined_audio.write_audiofile(os.path.join(self.paths["audios"], "combined_audio.mp3"))

        return combined_audio
    
    def combined_video(self,duration=5): 

        # Charger la vidéo
        video_clip = self.random_video()
        if(video_clip is None):
            return None
        
        start_video = random.uniform(0, video_clip.duration - duration - 1)

        
        video_clip=video_clip.subclipped(start_video,start_video+duration)
        print(f"La vidéo commence à  {start_video} s")
        
        

        # Charger l'image
        image_clip = ImageClip(self.choose_background())
        if(image_clip is None):
            return None

        #Si il y a besoin de mettre au format TikTok
        image_clip = image_clip.resized((1080, 1080))

        # Redimensionner la vidéo pour être à la bonne taille
        video_clip=video_clip.with_effects([vfx.Resize((1080, 840))])
        
        
        # Définir la durée de l'image (identique à la vidéo)
        image_clip = image_clip.with_duration(video_clip.duration)
        
        image_clip = image_clip.with_position(("center", "top"))
        video_clip = video_clip.with_position(("center", "bottom"))

        # creer le dossier srt subtitles
        self.create_srt(duration)
        subtitles = self.combined_subtitles(duration)

        # Combiner les deux clips
        final_clip = CompositeVideoClip([image_clip, video_clip, subtitles], size=(video_clip.w, video_clip.h + image_clip.h))

        #final_clip.write_videofile(os.path.join(self.paths["final"], "output_video_test.mp4"), codec='libx264', fps=24)
        
        return final_clip 
    
    def combined_subtitles(self,duration=5):
        subtitles = SubtitlesClipMine(subtitles=os.path.join(self.paths["audios"], "subtitles.srt"), font="fonts\Super Cartoon.ttf", encoding="utf-8")
        subtitles = subtitles.with_position(("center", 800))
        
        return subtitles
    
    def combined_text(self,duration=5):
        # Create a text clip
        txt_clip:TextClip = TextClip(text=self.subject, color='white',font="fonts\Super Cartoon.ttf", font_size=70,size=(1080, None))
        txt_clip = txt_clip.with_position(("center", "top")).with_duration(duration-2).with_start(1)

        txt_clip = txt_clip.with_effects([vfx.FadeIn(1, initial_color=[0, 0, 0]),
                                        vfx.FadeOut(1, final_color=[0, 0, 0])])
        txt_clip = txt_clip.with_mask()

        return txt_clip
    
    def generate_video_final(self):

        # Charger les fichiers audio
        audio = self.combined_audio()
        if audio is None:
            print("pas de musique")
            return None
        
        # charger la vidéo
        final_clip = self.combined_video(audio.duration)
        if(final_clip is None):
            print("pb video")
            return None
        
        # charger la text
        txt_clip = self.combined_text(audio.duration)       
        

        #print(final_clip.size)  # Affiche (width, height) de la vidéo finale
        #final_clip.preview(fps=10)

        # Ajouter l'audio combiné à la vidéo
        final_clip.audio = audio 
        # Ajouter le texte
        final_clip = CompositeVideoClip([final_clip, txt_clip])
        
        final_clip.write_videofile(os.path.join(self.paths["final"], "output_video.mp4"), codec='libx264', fps=24)


        

    
    def __str__(self):
        return f"Project about {self.subject}"
    

<>:263: SyntaxWarning: invalid escape sequence '\S'
<>:270: SyntaxWarning: invalid escape sequence '\S'
<>:263: SyntaxWarning: invalid escape sequence '\S'
<>:270: SyntaxWarning: invalid escape sequence '\S'
C:\Users\DELL\AppData\Local\Temp\ipykernel_2524\118075475.py:263: SyntaxWarning: invalid escape sequence '\S'
  subtitles = SubtitlesClipMine(subtitles=os.path.join(self.paths["audios"], "subtitles.srt"), font="fonts\Super Cartoon.ttf", encoding="utf-8")
C:\Users\DELL\AppData\Local\Temp\ipykernel_2524\118075475.py:270: SyntaxWarning: invalid escape sequence '\S'
  txt_clip:TextClip = TextClip(text=self.subject, color='white',font="fonts\Super Cartoon.ttf", font_size=70,size=(1080, None))


## Video

In [15]:
def generate_audios(speech:Speech, output_audio_path:str):
    create_audio(speech.intro, os.path.join(output_audio_path, "intro.mp3"),speech.text["age"],speech.text["gender"])
    create_audio(speech.outro, os.path.join(output_audio_path, "outro.mp3"),speech.text["age"],speech.text["gender"])
    voice=create_audio(speech.history, os.path.join(output_audio_path, "history.mp3"),speech.text["age"],speech.text["gender"])
    print("\nVoice used : "+voice+"\n")
        

In [16]:
def generate_background(subject:str, output_background_path:str):
    create_background(subject,os.path.join(output_background_path, "image.png"))

In [17]:
def generate_background_google(subject:str, output_background_path:str):
    create_background_google(subject,output_background_path)

## Générer que l'audio avec les dirs 

In [18]:
"""
subject = "Freeze Corleone"
project = Project(subject)
speech = Speech(subject)
speech.print_speech()

generate_audios(speech,project.paths["audios"])
project.combined_audio()
"""

'\nsubject = "Freeze Corleone"\nproject = Project(subject)\nspeech = Speech(subject)\nspeech.print_speech()\n\ngenerate_audios(speech,project.paths["audios"])\nproject.combined_audio()\n'

## Générer que la vidéo avec les dirs

In [19]:
"""
subject = "John Lennon"
project = Project(subject)
speech = Speech(subject)
print(speech)

generate_background_google(subject,project.paths["background"])
project.combined_video(5)
"""


'\nsubject = "John Lennon"\nproject = Project(subject)\nspeech = Speech(subject)\nprint(speech)\n\ngenerate_background_google(subject,project.paths["background"])\nproject.combined_video(5)\n'

## Générer toute la vidéo 

In [21]:
persos = [
    
]


for perso in persos :
    subject = perso
    speech = Speech(subject)
    print(speech)
    project = Project(subject,speech)

    generate_audios(speech,project.paths["audios"])
    # Création d'image Chat GPT
    # generate_background(subject,project.paths["background"]) 
    generate_background_google(subject,project.paths["background"])
    project.generate_video_final()





In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
from moviepy.video.VideoClip import TextClip
from moviepy.video.tools.subtitles import SubtitlesClip # Sous Titres
import numpy as np

# Exemple d'utilisation de SubtitlesClipMine
if __name__ == "__main__":
    # Générateur personnalisé pour créer un TextClip à partir d'un texte

    # Chemin de la vidéo et des sous-titres
    video_path = "output.mp4"
    subtitles_path = "subtitles.srt"

    # Chargement de la vidéo
    video = VideoFileClip(video_path)
    video=video.subclipped(0,5)

    # Création du SubtitlesClipMine avec le générateur de texte
    subtitles = SubtitlesClipMine(subtitles=subtitles_path, font='fonts\Super Cartoon.ttf', encoding="utf-8")
    #subtitles=subtitles.with_position(("center", "bottom"))
    subtitles = subtitles.with_position(("center", 1080))
    # Combinaison des sous-titres avec la vidéo
    final_video = CompositeVideoClip([video, subtitles])

    # Sauvegarde de la vidéo finale avec sous-titres
    output_path = "output_with_subtitles.mp4"
    final_video.write_videofile(output_path, fps=video.fps, codec="libx264", audio_codec="aac")


<>:20: SyntaxWarning: invalid escape sequence '\S'
<>:20: SyntaxWarning: invalid escape sequence '\S'
C:\Users\DELL\AppData\Local\Temp\ipykernel_2524\3484224700.py:20: SyntaxWarning: invalid escape sequence '\S'
  subtitles = SubtitlesClipMine(subtitles=subtitles_path, font='fonts\Super Cartoon.ttf', encoding="utf-8")


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.9.106'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1080, 1920], 'bitrate': 262, 'fps': 24.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'handler_name': 'VideoHandler', 'encoder': 'Lavc61.31.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'handler_name': 'SoundHandler'}}], 'input_number': 0}], 'duration': 48.41, 'bitrate': 395, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1080, 1920], 'video_bitrate': 262, 'video_fps': 24.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 1, 'audio_fps': 44100, 'au

chunk:  74%|███████▎  | 1284/1744 [03:18<00:00, 934.27it/s, now=None]

MoviePy - Building video output_with_subtitles.mp4.
MoviePy - Writing audio in output_with_subtitlesTEMP_MPY_wvf_snd.mp4


chunk:  74%|███████▎  | 1284/1744 [03:19<00:00, 934.27it/s, now=None]

MoviePy - Done.
MoviePy - Writing video output_with_subtitles.mp4

